#Fake news classifier

In [119]:
import jovian

In [6]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import warnings 
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=DeprecationWarning)
%config InlineBackend.figure_format='retina'
import io
from google.colab import files
from sklearn.model_selection import train_test_split

uploaded = files.upload()

Saving test.csv to test.csv
Saving train.csv to train.csv


In [120]:
Train=pd.read_csv('train.csv')
Train.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [121]:
Test=pd.read_csv('test.csv')
Test.head()

,id,title,author,text
0,20800,"Specter of Trump Loosens Tongues, if Not Purse...",David Streitfeld,"PALO ALTO, Calif. — After years of scorning..."
1,20801,Russian warships ready to strike terrorists ne...,NaN,Russian warships ready to strike terrorists ne...
2,20802,#NoDAPL: Native American Leaders Vow to Stay A...,Common Dreams,Videos #NoDAPL: Native American Leaders Vow to...
3,20803,"Tim Tebow Will Attempt Another Comeback, This ...",Daniel Victor,"If at first you don’t succeed, try a different..."
4,20804,Keiser Report: Meme Wars (E995),Truth Broadcast Network,42 mins ago 1 Views 0 Comments 0 Likes 'For th...


In [122]:
###Drop Nan Values
Train=Train.dropna()
Test=Test.dropna()

In [123]:
X=Train.drop('label',axis=1)
y=Train['label']

In [124]:
X.shape

(18285, 4)

In [125]:
y.shape

(18285,)

In [126]:
### Vocabulary size
voc_size=1000

#Onehot Representation

In [127]:
messages=X.copy()

In [17]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [128]:
messages.reset_index(inplace=True)

In [129]:
import nltk
import re
from nltk.corpus import stopwords

In [130]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [131]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    #print(i)
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [132]:
corpus[0:10]

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag']

In [133]:
from tensorflow.keras.preprocessing.text import one_hot
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr[0:10]

[[553, 165, 483, 9, 797, 343, 522, 775, 180, 539],
 [659, 470, 353, 955, 245, 447, 119],
 [82, 161, 534, 827],
 [106, 397, 457, 42, 450, 908],
 [344, 245, 345, 498, 308, 182, 245, 786, 968, 64],
 [408, 50, 326, 523, 558, 231, 456, 742, 889, 344, 345, 529, 74, 875, 119],
 [24, 76, 356, 668, 407, 388, 86, 127, 181, 732, 868],
 [595, 406, 183, 677, 383, 522, 231, 536, 181, 732, 868],
 [470, 492, 114, 534, 207, 641, 197, 365, 231, 332],
 [54, 351, 343, 155, 251, 43, 102, 97]]

#**Padding**

In [29]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
sent_length=25
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[  0   0   0 ... 775 180 539]
 [  0   0   0 ... 245 447 119]
 [  0   0   0 ... 161 534 827]
 ...
 [  0   0   0 ... 181 732 868]
 [  0   0   0 ... 284 916 810]
 [  0   0   0 ... 192  75 521]]


In [134]:
embedded_docs[0:5]

array([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0, 553, 165, 483,   9, 797, 343, 522, 775, 180, 539],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0, 659, 470, 353, 955, 245, 447, 119],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,  82, 161, 534, 827],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0, 106, 397, 457,  42, 450, 908],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0, 344, 245, 345, 498, 308, 182, 245, 786, 968,  64]],
      dtype=int32)

#Model Creation

In [135]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout


In [137]:
## Creating model
embedding_vector_features=50
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(120))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_10 (Embedding)     (None, 25, 50)            50000     
_________________________________________________________________
lstm_11 (LSTM)               (None, 120)               82080     
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 121       
Total params: 132,201
Trainable params: 132,201
Non-trainable params: 0
_________________________________________________________________
None


In [138]:
len(embedded_docs),y.shape

(18285, (18285,))

In [139]:
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [140]:
X_final.shape,y_final.shape

((18285, 25), (18285,))

In [141]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.3, random_state=42)

In [142]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=5,batch_size=128)

Epoch 1/5
100/100 [==============================] - 11s 88ms/step - loss: 0.5532 - accuracy: 0.7115 - val_loss: 0.2466 - val_accuracy: 0.8970
Epoch 2/5
100/100 [==============================] - 8s 84ms/step - loss: 0.2005 - accuracy: 0.9150 - val_loss: 0.2452 - val_accuracy: 0.9014
Epoch 3/5
100/100 [==============================] - 8s 83ms/step - loss: 0.1796 - accuracy: 0.9246 - val_loss: 0.2281 - val_accuracy: 0.9032
Epoch 4/5
100/100 [==============================] - 8s 83ms/step - loss: 0.1523 - accuracy: 0.9393 - val_loss: 0.2246 - val_accuracy: 0.9056
Epoch 5/5
100/100 [==============================] - 8s 83ms/step - loss: 0.1437 - accuracy: 0.9433 - val_loss: 0.2424 - val_accuracy: 0.9010


#Performance Metrics 

In [143]:
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score
y_predt=model.predict_classes(X_train)
print("Accuracy of training data", accuracy_score(y_train,y_predt))
print("Confusion matrix on training\n", confusion_matrix(y_train,y_predt))
print("cost of training model\n",classification_report(y_train,y_predt))

Accuracy of training data 0.9558559262442379
Confusion matrix on training
 [[6918  336]
 [ 229 5316]]
cost of training model
               precision    recall  f1-score   support

           0       0.97      0.95      0.96      7254
           1       0.94      0.96      0.95      5545

    accuracy                           0.96     12799
   macro avg       0.95      0.96      0.96     12799
weighted avg       0.96      0.96      0.96     12799



In [144]:
y_pred=model.predict_classes(X_test)
print("Accuracy on test data",accuracy_score(y_test,y_pred))
print("Accuracy on test data\n",confusion_matrix(y_test,y_pred))
print("cost of test data model\n",classification_report(y_test,y_pred))

Accuracy on test data 0.9010207801676996
Accuracy on test data
 [[2808  299]
 [ 244 2135]]
cost of test data model
               precision    recall  f1-score   support

           0       0.92      0.90      0.91      3107
           1       0.88      0.90      0.89      2379

    accuracy                           0.90      5486
   macro avg       0.90      0.90      0.90      5486
weighted avg       0.90      0.90      0.90      5486



In [145]:
import jovian
jovian.commit(project='2_NLP_Fake_News_Classifier.ipynb')

[jovian] Detected Colab notebook...


[jovian] Error: jovian.commit doesn't work on Colab unless the notebook was created and executed from Jovian.
Make sure to run the first code cell at the top after executing from Jovian.
Alternatively, you can download this notebook and upload it manually to Jovian.
Learn more: https://jovian.ai/docs/user-guide/run.html#run-on-colab
